# CSCE 636: Deep Learning
# UIN: 828002068
# Name: Mamoon Masud
# Submission - 8 
# Architecture - Transfer Learning ( Dense Net )
# Activity Recognition Project - Test Set Evaluation Code

In [1]:
import tensorflow
import os
import cv2
import keras
from tensorflow.keras.models import Sequential, Model
from keras.layers import Input, BatchNormalization, Activation, Conv3D, Dropout, Concatenate, AveragePooling3D, MaxPooling3D, Dense, Flatten, GlobalAveragePooling2D
from keras.activations import linear, softmax
from keras.applications import densenet
from tensorflow.keras import backend
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import (Conv2D, MaxPooling3D, Conv3D,MaxPooling2D)
from tensorflow.keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import keras.backend as K
import traceback


Using TensorFlow backend.


# Loading the Model From the saved JSON File

In [2]:
# Set the path to the saved location of the JSON file. 
base_path = 'path of the trained model architecture file'
model_path = os.path.join(base_path,'ver_9_archi.json' )
with open(model_path, 'r') as json_file:
    json_savedModel= json_file.read()
#load the model architecture 
model_j = tensorflow.keras.models.model_from_json(json_savedModel)
model_j.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
2d_input (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           2d_input[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [3]:
# Load the saved model weight
weight_path = os.path.join(base_path, 'T3D_weights.hdf5')
model_j.load_weights(weight_path)

# Defining Video Generator for Test Set

In [ ]:
FRAMES_PER_VIDEO = 20
FRAME_HEIGHT = 256
FRAME_WIDTH = 256
FRAME_CHANNEL = 3
NUM_CLASSES = 2
BATCH_SIZE = 5
EPOCHS = 100
#MODEL_FILE_NAME = '/content/drive/My Drive/DL Proj/Sub_9/partly_trained.h5'

input_shape = (FRAME_HEIGHT,FRAME_WIDTH,FRAME_CHANNEL)
nb_classes = 2
sample_input = np.empty([FRAMES_PER_VIDEO, FRAME_HEIGHT, FRAME_WIDTH, FRAME_CHANNEL], dtype=np.uint8)
#model  = densenet161_3D_DropOut(sample_input.shape, nb_classes)

In [ ]:
import os
import cv2
import numpy as np
from keras.utils.np_utils import to_categorical
import random

ROOT_PATH = 'Data Set path'

def get_video_frames(src, fpv, frame_height, frame_width):
    # print('reading video from', src)
    cap = cv2.VideoCapture(src)

    frames = []
    if not cap.isOpened():
        cap.open(src)
    ret = True
    while(True and ret):
        # Capture frame-by-frame
        ret, frame = cap.read()

        frames.append(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything done, release the capture
    cap.release()

    rnd_idx = random.randint(5,len(frames)-5)
    rnd_frame = frames[rnd_idx]
    rnd_frame = cv2.resize(rnd_frame,(224,224)) #Needed for Densenet121-2d

    # Return fpv=10 frames
    step = len(frames)//fpv
    avg_frames = frames[::step]
    avg_frames = avg_frames[:fpv]
    avg_resized_frames = []
    for af in avg_frames:
        rsz_f = cv2.resize(af, (frame_width, frame_height))
        avg_resized_frames.append(rsz_f)
    return np.asarray(rnd_frame)/255.0,np.asarray(avg_resized_frames)


def get_video_and_label(index, data, frames_per_video, frame_height, frame_width):
    # Read clip and appropiately send the sports' class
    frame, clip = get_video_frames(os.path.join(
        ROOT_PATH, data['path'].values[index].strip()), frames_per_video, frame_height, frame_width)
    sport_class = data['class'].values[index]

    frame = np.expand_dims(frame, axis=0)
    clip = np.expand_dims(clip, axis=0)

    return frame, clip, sport_class


def video_gen(data, frames_per_video, frame_height, frame_width, channels, num_classes, batch_size=4):
    while True:
        # Randomize the indices to make an array
        indices_arr = np.random.permutation(data.count()[0])
        for batch in range(0, len(indices_arr), batch_size):
            # slice out the current batch according to batch-size
            current_batch = indices_arr[batch:(batch + batch_size)]

            # initializing the arrays, x_train and y_train
            clip = np.empty([0, frames_per_video, frame_height, frame_width, channels], dtype=np.float32)
            frame = np.empty([0, 224, 224, 3], dtype=np.float32)

            y_train = np.empty([0], dtype=np.int32)

            for i in current_batch:
                # get frames and its corresponding color for an traffic light
                single_frame, single_clip, sport_class = get_video_and_label(
                    i, data, frames_per_video, frame_height, frame_width)

                # Appending them to existing batch
                frame = np.append(frame, single_frame, axis=0)
                clip = np.append(clip, single_clip, axis=0)

                y_train = np.append(y_train, [sport_class])
            y_train = to_categorical(y_train, num_classes=num_classes)

            yield ([frame, clip], y_train)

In [ ]:
bs_path = 'Data Set Path'
d_test = pd.read_csv(os.path.join(bs_path, 'test.csv'))


In [ ]:
video_test_generator = video_gen(
    d_test, FRAMES_PER_VIDEO, FRAME_HEIGHT, FRAME_WIDTH, FRAME_CHANNEL, nb_classes, batch_size=BATCH_SIZE)

sample_input = np.empty(
    [FRAMES_PER_VIDEO, FRAME_HEIGHT, FRAME_WIDTH, FRAME_CHANNEL], dtype=np.uint8)
  

In [ ]:
optim = Adam(lr=1e-4, decay=1e-6)
#optim = SGD(lr = 0.1, momentum=0.9, decay=1e-4, nesterov=True)
model_j.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
test_steps = len(d_test)//BATCH_SIZE

In [50]:
# 76th Epoch Trained weights
model_j.evaluate(video_test_generator, 
                 steps = test_steps,
                 verbose = 1 )

7/7 [==============================] - 79s 11s/step - loss: 0.2464 - accuracy: 0.9062


[0.24642086029052734, 0.90625]